<a href="https://colab.research.google.com/github/ofer1992/dl-notes/blob/master/fastai_dl1/lesson2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>